In [1]:
''' Notebook for creating VAE liver dataset. Therefore labels are not included. '''
import torch
import torch.nn.functional as F
import torch.utils as U
import torchvision.transforms as T
from torch import Tensor
import json
import nibabel as nib
import matplotlib.pyplot as plt
import numpy as np

In [2]:
organ = 'pancreas'
path = f'../raw_data/{organ}/dataset.json'
with open(path) as f:
    data_set = json.load(f)
training_paths = data_set['training']

In [3]:
d = 256 # width and height to resize to
#p1 = int(d/4)
#p2 = int(d/8)
#p3 = int(d/16)
resize_transform = T.Resize((d, d))
#pad_transforms = [T.Pad(p1), T.Pad(p2), T.Pad(p3)]
rand_rot_transform = T.RandomRotation(180) 

In [20]:
data_tensor = torch.zeros((0, d, d))
total_paths = len(training_paths)

for n, path in enumerate(training_paths):
    print(f'Iteration {n}/{total_paths}')
    nii_img = nib.load(f'../raw_data/{organ}' + path['image'])
    nii_data = nii_img.get_fdata()
    nii_data = Tensor(nii_img.get_fdata())
    # ensure scale [0; 1]
    nii_data -= nii_data.min()
    nii_data /= nii_data.max() # are the max the same in every data point?
    #nii_data -= 0.5
    #nii_data *= 2
    nii_data = nii_data.permute(2, 0, 1) # ensure the tensor shape is (slice, rows, columns)
    nii_data = resize_transform(nii_data)
    data_tensor = torch.cat((data_tensor, nii_data), 0)

Iteration 0/281
Iteration 1/281
Iteration 2/281
Iteration 3/281
Iteration 4/281
Iteration 5/281
Iteration 6/281
Iteration 7/281
Iteration 8/281
Iteration 9/281
Iteration 10/281
Iteration 11/281
Iteration 12/281
Iteration 13/281
Iteration 14/281
Iteration 15/281
Iteration 16/281
Iteration 17/281
Iteration 18/281
Iteration 19/281
Iteration 20/281
Iteration 21/281
Iteration 22/281
Iteration 23/281
Iteration 24/281
Iteration 25/281
Iteration 26/281
Iteration 27/281
Iteration 28/281
Iteration 29/281
Iteration 30/281
Iteration 31/281
Iteration 32/281
Iteration 33/281
Iteration 34/281
Iteration 35/281
Iteration 36/281
Iteration 37/281
Iteration 38/281
Iteration 39/281
Iteration 40/281
Iteration 41/281
Iteration 42/281
Iteration 43/281
Iteration 44/281
Iteration 45/281
Iteration 46/281
Iteration 47/281
Iteration 48/281
Iteration 49/281
Iteration 50/281
Iteration 51/281
Iteration 52/281
Iteration 53/281
Iteration 54/281
Iteration 55/281
Iteration 56/281
Iteration 57/281
Iteration 58/281
Iterati

In [21]:
torch.save(data_tensor, f'../augmenteddata/{organ}_slices_unaugmented.pt')

In [4]:
organs = ['pancreas', 'spleen', 'colon', 'lung', 'liver']

for n in range(10):
    chunk = torch.zeros(0, d, d)
    torch.save(chunk, f'../augmenteddata/unaugmented_chunk_{n}.pt')
# splits data up randomly into 10 chunks - all with equal parts of each organ
for i, organ in enumerate(organs):
    data = torch.load(f'../augmenteddata/{organ}_slices_unaugmented.pt')
    N = data.shape[0]
    idx = torch.randperm(N)
    data = data[idx]
    split_idx = int(N/10) # lazy way to split the data into 10 chunks - if the data is not divisible by 10, some data will be lost
    for n in range(10):
        chunk = torch.load(f'../augmenteddata/unaugmented_chunk_{n}.pt')
        chunk = torch.cat((chunk, data[n*split_idx:(n+1)*split_idx]), 0)
        torch.save(chunk, f'../augmenteddata/unaugmented_chunk_{n}.pt')